In [ ]:
import copy
import csv
import os
import warnings
from argparse import ArgumentParser
import numpy as np

import torch
from tqdm import tqdm
import yaml
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
import glob
import json
import os
from nets import ContextNn
from utils import util
import pandas as pd
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
with open('utils/args.yaml', errors='ignore') as f:
    params = yaml.safe_load(f) 

In [ ]:
def crop_image_and_label(crop_h,crop_w,label,input_size):
    temp_labels = []
    for k in range(len(label)):
        x = label[k][2]
        y = label[k][1]
        w = label[k][4]
        h = label[k][3]
        if x >= crop_h and y >= crop_w and x <= crop_h + input_size and y <= crop_w + input_size:
            label[k][1] = (y+h/2 - crop_w)/ input_size
            label[k][2] = (x+w/2 - crop_h)/ input_size
            label[k][3] = (h) / input_size
            label[k][4] = (w) / input_size
            temp_labels.append(label[k])
    return temp_labels
    
def find_key_by_value(dictionary, target_value):
    """value로 key 찾기"""
    for key, value in dictionary.items():
        if value == target_value:
            return key
    return None 

input_size = 512
label_dir = '../../data/Brest_spatialTranscriptome/preprocessed_xenium/patch_train_data/**/annotation/'
label_files = sorted(glob.glob(f"{label_dir}/*.csv"))
image_filenames = []
tissue_image_filenames = []
labels = []
# 라벨이 존재하는 이미지
for label_file in label_files:
    data1 = pd.read_csv(label_file)

    img_path = label_file.replace('/annotation', '/image').replace('.csv', '.png')
    tissue_image_path=label_file.replace('/annotation', '/tissue_image').replace('.csv', '.png')
    if os.path.exists(img_path):
        image_filenames.append(img_path)
        tissue_image_filenames.append(tissue_image_path)
        arr = data1.to_numpy()
        arr=arr[:, [4, 1, 0 , 3, 2]]#class, y, x, h, w
        temp_labels=list(arr)
        labels.append(temp_labels)

images = []
tissue_image=[]
for i in tqdm(range(len(image_filenames))):
    image =cv2.imread(image_filenames[i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
    tissue_img = cv2.imread(tissue_image_filenames[i])
    tissue_img = cv2.cvtColor(tissue_img, cv2.COLOR_BGR2RGB)  # BGR -> RGB 변환
    tissue_image.append(tissue_img)
    images.append(image)


In [ ]:
def collate_fn1(batch):
    """Tissue context를 포함한 배치 collation"""
    samples, context_samples, cls, box, indices = zip(*batch)

    cls = torch.cat(cls, dim=0)
    box = torch.cat(box, dim=0)

    new_indices = list(indices)
    for i in range(len(indices)):
        new_indices[i] += i
    indices = torch.cat(new_indices, dim=0)

    targets = {'cls': cls,
                'box': box,
                'idx': indices}
    return torch.stack(samples, dim=0), torch.stack(context_samples, dim=0), targets

class custom_dataset(data.Dataset):
    def __init__(self, images, params, augment, labels, image_infos=None):
        self.params = params
        self.mosaic = augment
        self.augment = augment
        self.images = images[0]
        self.tissue_images = images[1]
        self.labels = labels
        self.scale_list = [256, 288, 320, 352, 384, 416, 448, 480, 512]
        self.current_scale = 512  # 배치 단위로 관리될 현재 스케일
        self.n = len(self.images)
        self.indices = list(range(self.n))
    
    def __len__(self):
        return len(self.indices)
    
    def set_scale(self, scale):
        """배치 단위로 스케일 설정"""
        self.current_scale = scale
    
    def random_multi_scale(self, image, label):
        # 현재 설정된 스케일 사용 (배치 내 모든 이미지가 동일한 크기)
        size = self.current_scale
        image = cv2.resize(image, (size, size))
        temp_labels = []
        for i in range(len(label)):
            x_center = label[i][1]
            y_center = label[i][2]
            w = label[i][3]
            h = label[i][4]
            temp_labels.append([label[i][0], y_center, x_center, h, w])
        return image, temp_labels
    
    def __getitem__(self, index):
        
        index = self.indices[index]
        image = self.images[index].copy()
        tissue_image = self.tissue_images[index].copy()
        label = self.labels[index]
        
        image,label=self.random_multi_scale(image,label)
        cls = []
        box = []
        for i in range(len(label)):
            cls.append(label[i][0])
            box.append(label[i][1:5])
        cls = np.array(cls)
        box = np.array(box)
        nl = len(box)
        
        if self.augment:
            # Flip up-down (메인 이미지와 tissue context 동시 적용)
            if random.random() < self.params['flip_ud']:
                image = np.flipud(image).copy()
                tissue_image = np.flipud(tissue_image).copy()
                if nl:
                    box[:, 1] = 1 - box[:, 1]
            
            # Flip left-right (메인 이미지와 tissue context 동시 적용)
            if random.random() < self.params['flip_lr']:
                image = np.fliplr(image).copy()
                tissue_image = np.fliplr(tissue_image).copy()
                if nl:
                    box[:, 0] = 1 - box[:, 0]

        # HWC -> CHW 변환
        image = image.transpose((2, 0, 1))
        tissue_image = tissue_image.transpose((2, 0, 1))
        
        return (torch.from_numpy(image), 
                torch.from_numpy(tissue_image),  # tissue context 추가
                torch.from_numpy(cls), 
                torch.from_numpy(box), 
                torch.zeros(nl))
    

split=[0.9, 0.1]
patch_train, patch_test, tissue_train, tissue_test, label_train, label_test = train_test_split(
    images, tissue_image, labels, 
    test_size=0.15,
    random_state=242,
    shuffle=True
)

# Dataset 생성
train_dataset = custom_dataset((patch_train, tissue_train), params, augment=True, labels=label_train)
val_dataset = custom_dataset((patch_test, tissue_test), params, augment=False, labels=label_test)

In [ ]:
def visualize_sample_with_overlay(dataset, index=0):
    """
    데이터셋의 특정 샘플에 대해 바운딩 박스를 오버레이한 이미지를 시각화합니다.
    메인 이미지와 tissue context 이미지를 함께 표시합니다.
    
    Args:
        dataset: 커스텀 데이터셋
        index: 시각화할 샘플의 인덱스
    """
    # 샘플 가져오기 (tissue context 포함)
    image_tensor, tissue_tensor, cls_tensor, box_tensor, _ = dataset[index]
    
    # 텐서를 numpy 배열로 변환
    # 이미지: (C, H, W) -> (H, W, C)
    image = image_tensor.numpy().transpose(1, 2, 0)
    tissue_image = tissue_tensor.numpy().transpose(1, 2, 0)
    cls = cls_tensor.numpy()
    boxes = box_tensor.numpy()

    # 이미지 크기
    height, width = image.shape[:2]
    
    # 클래스 이름 및 색상 설정
    class_names = {
    0: "epithelial",
    1: "Basal/Myoepithelial",
    2: "Smooth muscle",
    3: "Fibroblast",
    4: "Endothelial",
    5: "Lymphocyte",                # T + B 통합
    6: "Plasma cell",
    7: "Macrophage/Histiocyte",     # 통합
    8: "Neutrophil",
    9: "Adipocyte",
    10: "Other/Unknown"
}
    colors = ["#FF0000","#FFA500",
    "#8B4513",
    "#00FF00",
    "#0000FF",
    "#FFFF00",
    "#FF00FF",
    "#9400D3",
    "#00FFFF",
    "#FF6060",
    "#808080"]

    
    # 시각화: 메인 이미지와 tissue context를 나란히 표시
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    
    # 메인 이미지
    axes[0].imshow(image.astype(np.uint8))
    for i in range(len(boxes)):
        center_x, center_y, box_width, box_height = boxes[i]
        center_x_pixel = center_x * width
        center_y_pixel = center_y * height
        box_w_pixel = box_width * width
        box_h_pixel = box_height * height
        
        # 좌상단 좌표 계산 (중심점 -> 좌상단)
        x1 = center_x_pixel - box_w_pixel / 2
        y1 = center_y_pixel - box_h_pixel / 2
        
        class_id = int(cls[i])
        color = colors[class_id] if class_id < len(colors) else 'green'
        
        # Bounding box 그리기
        rect = patches.Rectangle((x1, y1), box_w_pixel, box_h_pixel, 
                                 linewidth=2, edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)
    
    axes[0].set_title(f'Main Image - Sample {index}', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # Tissue Context 이미지
    axes[1].imshow(tissue_image.astype(np.uint8))
    axes[1].set_title(f'Tissue Context - Sample {index}', fontsize=12, fontweight='bold')
    axes[1].axis('off')
    
    fig.suptitle(f'Dataset Sample {index}\nTotal Objects: {len(boxes)} | '
                f'Class Distribution: {[f"{i}:{sum(cls==i)}" for i in range(12)]}',
                fontsize=14, fontweight='bold')
    
    # 범례 추가
    legend_elements = [
        patches.Patch(color=colors[i], label=class_names[i]) for i in range(len(colors))
    ]
    fig.legend(handles=legend_elements, loc='lower center', ncol=6, 
               bbox_to_anchor=(0.5, -0.05), fontsize=9)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.12)
    plt.show()
    
    # 상세 정보 출력
    print(f"📊 Sample {index} 상세 정보:")
    print(f"  메인 이미지 크기: {width} x {height}")
    print(f"  Tissue context 크기: {tissue_image.shape[1]} x {tissue_image.shape[0]}")
    print(f"  총 객체 수: {len(boxes)}")
    print(f"  클래스별 분포:")
    for class_id in range(12):
        count = sum(cls == class_id)
        if count > 0:
            print(f"    Class {class_id}: {count}개")
    
    print("\n🔍 처음 5개 객체 정보:")
    for i in range(min(5, len(boxes))):
        center_x, center_y, box_w, box_h = boxes[i]
        class_id = int(cls[i])
        class_name = class_names.get(class_id, f'Class {class_id}')
        print(f"  객체 {i+1}: {class_name} | 중심점: ({center_x:.3f}, {center_y:.3f}) | 크기: {box_w:.3f} x {box_h:.3f}")

# train_dataset 시각화 실행
print("🖼️ Train Dataset 샘플 시각화 (메인 이미지 + Tissue Context)")
print("=" * 70)
visualize_sample_with_overlay(train_dataset, index=14)

In [ ]:
plt.imshow(train_dataset[14][0].numpy().transpose(1, 2, 0))

In [ ]:


# 모델 및 파라미터 준비 (Tissue context 활성화)
model = ContextNn.yolo_v11_m(len(params['names']), use_context=True, context_backbone='efficientnet_b1').to(device)
optimizer = torch.optim.AdamW(util.set_params(model, params['weight_decay']),
                             lr=2e-4, 
                             betas=(0.9, 0.999),
                             weight_decay=params['weight_decay'])
criterion = util.ComputeLoss(model, params)

# 데이터셋 및 데이터로드 (안전한 함수 사용)
batch_size = 32
# 안전하게 데이터로더 생성하는 함수
def create_safe_loader(dataset, batch_size, is_train=True):
    """
    배치 크기에 맞게 데이터셋을 조정하여 안전하게 데이터로더를 생성하는 함수
    """
    dataset_size = len(dataset)
    
    # 배치 크기가 데이터셋 크기보다 큰 경우 배치 크기 조정
    if dataset_size < batch_size:
        print(f"경고: 데이터셋 크기({dataset_size})가 배치 크기({batch_size})보다 작습니다. 배치 크기를 {dataset_size}로 조정합니다.")
        actual_batch_size = max(1, dataset_size)
    else:
        actual_batch_size = batch_size
    
    # 데이터셋이 배치 크기로 나누어 떨어지는지 확인
    if dataset_size % actual_batch_size != 0:
        print(f"참고: 데이터셋 크기({dataset_size})가 배치 크기({actual_batch_size})로 나누어 떨어지지 않습니다.")
        print(f"마지막 배치는 {dataset_size % actual_batch_size}개의 샘플을 포함합니다.")
    
    # 데이터로더 생성
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=actual_batch_size, 
        shuffle=is_train,
        num_workers=4,
        collate_fn=collate_fn1,
        drop_last=(not is_train)  # 훈련 시에는 마지막 배치 유지, 검증 시에는 마지막 배치 제외
    )
    
    return loader, actual_batch_size
# 안전하게 데이터로더 생성
loader, train_batch_size = create_safe_loader(train_dataset, batch_size, is_train=True)
val_loader, val_batch_size = create_safe_loader(val_dataset, batch_size, is_train=False)

print(f"최종 훈련 배치 크기: {train_batch_size}")
print(f"최종 검증 배치 크기: {val_batch_size}")



In [ ]:
from utils.valid import compute_point_label_metrics  # Point-label 최적화 메트릭
from utils.valid import visualize_ground_truth_and_prediction_separately
from utils.valid import plot_training_progress


# main.py의 train 함수를 참조한 개선된 학습 루프
train_losses = []

# Point-label 메트릭용 리스트
val_det_recalls = []  # Detection Recall
val_cls_accs = []     # Classification Accuracy
val_macro_f1s = []    # Macro F1-score
val_macro_precisions = []  # Macro Precision
val_macro_recalls = []  # Macro Recall

# 클래스별 통계 저장용 리스트 (각 에폭마다 class_stats dict 저장)
val_class_stats_history = []

epochs = 10000
# scale_list = [256, 288, 320, 352, 384, 416, 448, 480, 512]
scale_list = [512]
# 체크포인트 저장을 위한 디렉토리 생성
save_dir = '../../model/Breast_ST_context/'
os.makedirs(save_dir, exist_ok=True)
#체크포인트 불러오기 
checkpoint_path = os.path.join(save_dir, 'last_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
# main.py 스타일의 설정들
best_macro_Overall_Recall = 0  # 베스트 모델 기준: macro_f1
accumulate = max(round(64 / batch_size), 1)  # gradient accumulation steps
amp_scale = torch.amp.GradScaler()  # mixed precision scaler

print(f"Gradient accumulation steps: {accumulate}")

for epoch in range(epochs):
    # 훈련
    model.train()
    
    # main.py 스타일의 평균 손실 추적
    avg_box_loss = util.AverageMeter()
    avg_cls_loss = util.AverageMeter()
    avg_dfl_loss = util.AverageMeter()
    
    train_pbar = tqdm(enumerate(loader), total=len(loader), desc=f'Epoch {epoch+1}/{epochs} Training')
    
    optimizer.zero_grad()
    current_scale = random.choice(scale_list)
    train_dataset.set_scale(current_scale)
    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        collate_fn=collate_fn1,
        drop_last=True
    )
    for i, (torch_images, tissue_images, targets) in train_pbar:
        step = i + len(loader) * epoch
        torch_images = torch_images.to(device).float() / 255.
        tissue_images = tissue_images.to(device).float() / 255.
        
        # Forward pass with mixed precision (tissue context 포함)
        with torch.amp.autocast('cuda'):
            outputs = model(torch_images, tissue_context=tissue_images)
            loss_box, loss_cls, loss_dfl = criterion(outputs, targets)
        
        # 평균 손실 업데이트
        avg_box_loss.update(loss_box.item(), torch_images.size(0))
        avg_cls_loss.update(loss_cls.item(), torch_images.size(0))
        avg_dfl_loss.update(loss_dfl.item(), torch_images.size(0))
        
        # Loss scaling for gradient accumulation
        loss_box *= batch_size  # loss scaled by batch_size
        loss_cls *= batch_size  # loss scaled by batch_size  
        loss_dfl *= batch_size  # loss scaled by batch_size
        
        total_loss = loss_box + loss_cls + loss_dfl
        
        # Backward pass with gradient scaling
        amp_scale.scale(total_loss).backward()
        
        # Gradient accumulation 및 optimization
        if step % accumulate == 0:
            # Gradient clipping 및 optimization
            amp_scale.step(optimizer)
            amp_scale.update()
            optimizer.zero_grad()
        
        # GPU 메모리 동기화
        torch.cuda.synchronize()
        
        # 진행률 표시 업데이트 (main.py 스타일)
        memory = f'{torch.cuda.memory_reserved() / 1E9:.4g}G'
        s = f'Memory: {memory} | Box: {avg_box_loss.avg:.3f} | Cls: {avg_cls_loss.avg:.3f} | DFL: {avg_dfl_loss.avg:.3f}'
        train_pbar.set_description(f'Epoch {epoch+1}/{epochs} | {s}')
    
    # 에폭 평균 손실 계산
    avg_train_loss = avg_box_loss.avg + avg_cls_loss.avg + avg_dfl_loss.avg
    train_losses.append(avg_train_loss)
    val_dataset.set_scale(512)
    
    # Point-label 최적화 메트릭 계산 (주요 평가 지표)
    point_metrics = compute_point_label_metrics(
        model, val_loader, device, params, distance_threshold=16
    )
    
    detection_recall = point_metrics.get('detection_recall', 0)
    cls_accuracy = point_metrics.get('classification_accuracy', 0)
    macro_precision = point_metrics.get('macro_precision', 0)
    macro_recall = point_metrics.get('macro_recall', 0)
    macro_f1 = point_metrics.get('macro_f1', 0)
    overall_recall = point_metrics.get('overall_recall', 0)
    class_stats = point_metrics.get('class_stats', {})
    
    val_det_recalls.append(detection_recall)
    val_cls_accs.append(cls_accuracy)
    val_macro_f1s.append(macro_f1)
    val_macro_precisions.append(macro_precision)
    val_macro_recalls.append(macro_recall)
    
    # 클래스별 통계 저장
    val_class_stats_history.append(class_stats)
    
    # 결과 출력
    print(f"\nEpoch {epoch+1}/{epochs} Results:")
    print(f"  Train Loss - Box: {avg_box_loss.avg:.4f}, Cls: {avg_cls_loss.avg:.4f}, DFL: {avg_dfl_loss.avg:.4f}, Total: {avg_train_loss:.4f}")
    print(f"🎯 Point-Label Metrics:")
    print(f"  Detection Recall: {detection_recall:.4f} (GT 중 찾은 비율)")
    print(f"  Classification Accuracy: {cls_accuracy:.4f} (찾은 세포의 분류 정확도)")
    print(f"  Macro Precision: {macro_precision:.4f}")
    print(f"  Macro Recall: {macro_recall:.4f}")
    print(f"  Macro F1-score: {macro_f1:.4f} ⭐")
    print(f"  Overall Recall: {overall_recall:.4f}")
    
    
    # 베스트 모델 저장 (Macro F1 기준)
    if overall_recall > best_macro_Overall_Recall:
        best_macro_Overall_Recall = overall_recall
        save_checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'amp_scale_state_dict': amp_scale.state_dict(),
            'train_loss': avg_train_loss,
            'box_loss': avg_box_loss.avg,
            'cls_loss': avg_cls_loss.avg,
            'dfl_loss': avg_dfl_loss.avg,
            # Point-label 메트릭
            'detection_recall': detection_recall,
            'classification_accuracy': cls_accuracy,
            'macro_precision': macro_precision,
            'macro_recall': macro_recall,
            'macro_f1': macro_f1,
            'overall_recall': overall_recall,
            'class_stats': class_stats,
            'params': params
        }
        torch.save(save_checkpoint, os.path.join(save_dir, 'best_model.pt'))
        print(f"🎉 새로운 베스트 모델 저장! overall recall: {overall_recall:.4f}")
    
    # 최신 모델도 저장 (main.py 스타일)
    last_checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'amp_scale_state_dict': amp_scale.state_dict(),
        'train_loss': avg_train_loss,
        'box_loss': avg_box_loss.avg,
        'cls_loss': avg_cls_loss.avg,
        'dfl_loss': avg_dfl_loss.avg,
        # Point-label 메트릭
        'detection_recall': detection_recall,
        'classification_accuracy': cls_accuracy,
        'macro_precision': macro_precision,
        'macro_recall': macro_recall,
        'macro_f1': macro_f1,
        'overall_recall': overall_recall,
        'class_stats': class_stats,
        'params': params
    }
    torch.save(last_checkpoint, os.path.join(save_dir, 'last_model.pt'))
    
    # 100 에폭마다 학습 진행 그래프 생성 및 저장 (Point-label 메트릭 + 클래스별 성능)
    if (epoch + 1) % 100 == 0:
        try:
            print(f"\n📊 Epoch {epoch+1} - 학습 진행 상황 그래프 생성 중...")
            plot_training_progress(
                train_losses, val_det_recalls, val_cls_accs, val_macro_precisions, 
                val_macro_recalls, val_macro_f1s, epoch+1, save_dir, 
                class_stats_history=val_class_stats_history
            )
        except Exception as e:
            print(f"그래프 생성 중 오류: {e}")
    
    # 개선된 검증 이미지 시각화 (매 10 에폭마다) - 실제 라벨과 예측 라벨을 별도 figure로 표시
    if (epoch+1) % 10 == 0:
        try:
            # 여러 샘플에 대해 시각화
            num_samples = 1 # 샘플 수를 1개
            for sample_idx in range(num_samples):
                print(f"\n📊 Epoch {epoch+1} - 검증 샘플 {sample_idx+1}/{num_samples}:")
                print("=" * 60)
                
                # 실제 라벨과 예측 라벨을 별도 figure로 표시
                sample_idx = random.randint(0, len(val_dataset)-1)
                visualize_ground_truth_and_prediction_separately(
                    model, val_dataset, idx=sample_idx, 
                    epoch=epoch+1, save_dir=save_dir
                )
                
        except Exception as e:
            print(f"시각화 중 오류: {e}")

print("🎯 학습 완료!")
print(f"최종 베스트 Overall Recall: {best_macro_Overall_Recall:.4f}")
print(f"모델 저장 위치: {save_dir}")
print(f"베스트 모델: {os.path.join(save_dir, 'best_model.pt')}")
print(f"최신 모델: {os.path.join(save_dir, 'last_model.pt')}")

# 최종 성능 요약
if val_macro_f1s:
    final_macro_f1 = val_macro_f1s[-1]
    final_det_recall = val_det_recalls[-1]
    final_cls_acc = val_cls_accs[-1]
    final_macro_precision = val_macro_precisions[-1]
    final_macro_recall = val_macro_recalls[-1]
    
    print(f"\n📊 최종 성능 요약 (Point-Label Metrics):")
    print(f"  Macro F1-score: {final_macro_f1:.4f} ⭐")
    print(f"  Macro Precision: {final_macro_precision:.4f}")
    print(f"  Macro Recall: {final_macro_recall:.4f}")
    print(f"  Detection Recall: {final_det_recall:.4f}")
    print(f"  Classification Accuracy: {final_cls_acc:.4f}")


In [ ]:
from utils.valid import compute_point_label_metrics  # Point-label 최적화 메트릭
from utils.valid import visualize_ground_truth_and_prediction_separately
from utils.valid import plot_training_progress


# main.py의 train 함수를 참조한 개선된 학습 루프
train_losses = []

# Point-label 메트릭용 리스트
val_det_recalls = []  # Detection Recall
val_cls_accs = []     # Classification Accuracy
val_macro_f1s = []    # Macro F1-score
val_macro_precisions = []  # Macro Precision
val_macro_recalls = []  # Macro Recall

# 클래스별 통계 저장용 리스트 (각 에폭마다 class_stats dict 저장)
val_class_stats_history = []

epochs = 10000
# scale_list = [256, 288, 320, 352, 384, 416, 448, 480, 512]
scale_list = [512]
# 체크포인트 저장을 위한 디렉토리 생성
save_dir = '../../model/Breast_ST_context/'
os.makedirs(save_dir, exist_ok=True)
#체크포인트 불러오기 
checkpoint_path = os.path.join(save_dir, 'last_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
sample_idx = random.randint(0, len(val_dataset)-1)
visualize_ground_truth_and_prediction_separately(
                    model, val_dataset, idx=sample_idx, 
                    epoch=1, save_dir=save_dir,conf_threshold=0.1,
                    iou_threshold=0.1
                )

In [ ]:
# Cohen's Kappa 관련 함수들을 valid.py에서 import
from utils.valid import compute_validation_metrics_with_kappa, get_kappa_interpretation, quick_kappa_test

# 현재 모델의 빠른 Cohen's Kappa 테스트
print("🔍 현재 모델의 Cohen's Kappa 빠른 측정...")
current_kappa = quick_kappa_test(model, val_loader, device)